In [12]:

from dotenv import load_dotenv
import os
import sys
import json
import pandas as pd
import time
import datetime
load_dotenv()

path = os.path.abspath('')
module_path = os.path.join(path,'..\..')
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path+"\\functions")


from dataset_utils import DatasetEvaluator

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SQLAgents\..\..


<>:11: SyntaxWarning: invalid escape sequence '\.'
<>:11: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Tahsin\AppData\Local\Temp\ipykernel_8288\2745404199.py:11: SyntaxWarning: invalid escape sequence '\.'
  module_path = os.path.join(path,'..\..')


In [13]:
DEFAULT_PATH = "results/"
EXPERIMENT = "SQLAgent"
TARGET_PATH = f"evaluator_results/{EXPERIMENT}_results.csv"
TARGET_PATH_FK = f"evaluator_results/{EXPERIMENT}_foreign_key_results.csv"

Useful Functions

In [14]:
def save_queries(queries, FILE_NAME_PATH):
    data = {"queries":queries}
    with open(FILE_NAME_PATH, "w") as arquivo_json:
        json.dump(data, arquivo_json, indent=4) 

def read_queries(FILE_NAME_PATH):
    with open(FILE_NAME_PATH, encoding="utf-8", errors="ignore") as json_data:
        data = json.load(json_data, strict=False)
    queries = data["queries"]
    return queries

def get_schema(FILE_NAME_PATH):
    if "mondial_gpt" in FILE_NAME_PATH:
        return "mondial_gpt"
    elif "mondial" in FILE_NAME_PATH:
        return "mondial"
    return ""

def get_count_query_type(queries):
    count_types = {}
    for item in queries:
        type = item["type"]
        if type in count_types:
            count_types[type] += 1
        else:
            count_types[type] = 1
    count_types['total'] = len(queries)
    return count_types

def get_dataset_evaluator(FILE_NAME_PATH):
    SCHEMA = get_schema(FILE_NAME_PATH)  
    PREFIX = SCHEMA  
    if SCHEMA == "mondial_gpt":
        PREFIX = "mondial"
    print(PREFIX)
    print(SCHEMA)
    dataset_eval = DatasetEvaluator(dataset_file_path=f"../../datasets/{PREFIX}/{PREFIX}_dataset.json",
                                dataset_tables_path=f"../../datasets/{PREFIX}/result_tables/",
                                db_connection_file=f"../../datasets/{SCHEMA}_db_connection.json",
                                dataset_name=SCHEMA)
    return dataset_eval

def get_files_in_folder():
    files = []
    for r, d, f in os.walk(DEFAULT_PATH):
        for file in f:
            if '.json' in file and 'erro' not in file:
                files.append(os.path.join(r, file))
    return sorted(files)

def evaluate_query(queries, dataset_eval, FILE_NAME_PATH):
    result = {'simple': 0, 'medium': 0, 'complex': 0, 'total': 0}

    for q in queries:
        evaluate_result = False
        q['similarity'] = 0
        try:
            if q["query_string"].strip() != "":
                evaluate_result, similarity, column_matching_index = dataset_eval.evaluate_dataset_query(q["query_string"], q["id"], query_type="sql")
                q['column_matching_index'] = column_matching_index
                q['similarity']=similarity
            
        except Exception as e:
            print(str(e))
            
        q['result'] = evaluate_result
        
        if evaluate_result==True:
            result['total'] += 1
            result[q["type"]] += 1
        
    save_queries(queries, FILE_NAME_PATH)   
    return result

def get_file_name(FILE_NAME_PATH):
    parts = FILE_NAME_PATH.split("/")
    last_element = parts[-1]
    file_name = last_element.replace(".json", "")
    return file_name

def get_experiment_name(filename):
    schema = get_schema(filename).upper()
    experiment_name = f"{EXPERIMENT} - {schema}"
    if "chatgpt4" in filename:
        experiment_name += " - GPT-4"
    elif "chatgpt_" in filename:
        experiment_name += " - GPT-3.5"
    elif "llama" in filename:
        experiment_name += " - LLAMA"
        
    if "instances" in filename:
        experiment_name += " - Passing Database Rows"
        
    if "_fk" in filename:
        experiment_name += " - FK"
    return experiment_name


def compute_cost(filename):
    queries = read_queries(filename)
    total_costs = 0
    total_tokens = 0
    input_tokens = 0
    output_tokens = 0
    #total_token
    for q in queries:
        total_tokens += q.get('total_tokens', 0)
        total_costs += q.get('total_cost', 0)
        input_tokens += q.get('prompt_tokens', 0)
        output_tokens += q.get('completion_tokens', 0)

    print(total_tokens)
    print(round(total_costs, 2))
    print(input_tokens, output_tokens)
    return total_tokens, round(total_costs, 2), input_tokens, output_tokens

def compute_time(filename):
    queries = read_queries(filename)
    total_time = 0
    for q in queries:
        total_time += q.get('time', 0)
    print(total_time)
    return str(datetime.timedelta(seconds=round(total_time)))

In [15]:
def run(result_files = [], results = [], compute_costs = True):
    
    if result_files == [] :
        result_files = get_files_in_folder()
    else:
        result_files = sorted(result_files)
        
    
    for file in result_files:
        queries = read_queries(file)
        dataset_eval = get_dataset_evaluator(file)
        count_query_type = get_count_query_type(queries)
        result = evaluate_query(queries, dataset_eval, file)
        
        overall_by_type = {}
        for item in result:
            overall_by_type[f"overall_{item}"] = round(result[item] / count_query_type[item], 4)
        
        result['experiment'] = get_file_name(file)
        result = {**result, **overall_by_type}
        

        if compute_costs:
            result['total_tokens'], result['total_cost'], result['input_tokens'], result['output_tokens'] = compute_cost(file)
        result['total_time'] = compute_time(file)
        results.append(result)
        time.sleep(5)
    return results


In [16]:
compute_cost('results/7_sql_agent_chatgpt_mondial_gpt_fk.json')

0
0.0
0 0


(0, 0.0, 0, 0)

# Validação com os Esquemas com Foreign Keys

In [17]:
results_fk = run(['results/7_sql_agent_chatgpt_mondial_gpt_fk.json',
 'results/8_sql_agent_chatgpt_mondial_gpt_fk_instances.json'], [])

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SQLAgents\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SQLAgents\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SQLAgents\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI

mondial
mondial_gpt
{'DB_HOST': 'localhost', 'DB_PORT': '1522', 'DB_USER_NAME': 'MONDIAL_GPT', 'DB_PASS': 'TextDB123', 'DB_NAME': '', 'SQL_DRIVER': 'oracle+oracledb', 'SERVICE_NAME': 'xepdb1', 'SCHEMA': 'MONDIAL_GPT', 'KEYWORD_SEARCH_API_URL': ''}
SELECT max(area) FROM mondial_country where name  = 'Thailand'
SELECT name, area FROM country WHERE name = 'Thailand'
True cols: Index(['MAX(AREA)'], dtype='object')
Adj. True cols: Index(['MAX(AREA)'], dtype='object')
Predicted cols: Index(['NAME', 'AREA'], dtype='object')
Adj. Predicted cols: Index(['AREA'], dtype='object')
True: query matches sql (Similarity: 1.0, column match: 1.0)
SELECT name FROM mondial_province where area > 10000
SELECT name, area FROM province WHERE area > 10000;
True cols: Index(['NAME'], dtype='object')
Adj. True cols: Index(['NAME'], dtype='object')
Predicted cols: Index(['NAME', 'AREA'], dtype='object')
Adj. Predicted cols: Index(['NAME'], dtype='object')
True: query matches sql (Similarity: 1.0, column match: 1.

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SQLAgents\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SQLAgents\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SQLAgents\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI

True cols: Index(['NAME'], dtype='object')
Adj. True cols: Index([], dtype='object')
Predicted cols: Index(['NAME', 'POPULATION', 'ELEVATION', 'NAME'], dtype='object')
Adj. Predicted cols: Index([], dtype='object')
False: no match found
SELECT DISTINCT government FROM mondial_politics
SELECT DISTINCT government FROM politics
True cols: Index(['GOVERNMENT'], dtype='object')
Adj. True cols: Index(['GOVERNMENT'], dtype='object')
Predicted cols: Index(['GOVERNMENT'], dtype='object')
Adj. Predicted cols: Index(['GOVERNMENT'], dtype='object')
True: query matches sql (Similarity: 1.0, column match: 1.0)
SELECT p.name FROM mondial_province p INNER JOIN mondial_country c ON p.country = c.code  WHERE c.name = 'Germany'
SELECT name FROM province WHERE country = 'DE'
True cols: Index(['NAME'], dtype='object')
Adj. True cols: Index([], dtype='object')
Predicted cols: Index(['NAME'], dtype='object')
Adj. Predicted cols: Index([], dtype='object')
False: no match found
SELECT eg.percentage FROM mondia

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SQLAgents\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SQLAgents\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SQLAgents\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI

...

True cols: Index(['NUMBER_OF_COUNTRIES'], dtype='object')
Adj. True cols: Index([], dtype='object')
Predicted cols: Index(['NAME'], dtype='object')
Adj. Predicted cols: Index([], dtype='object')
False: no match found
SELECT SUM(DISTINCT c.area) AS total_area
FROM mondial_Country c
JOIN mondial_encompasses e ON c.code = e.country
JOIN mondial_geo_sea gs ON e.country = gs.country
WHERE gs.sea = 'Caribbean Sea';
SELECT c.name, c.area
FROM country c
JOIN borders b ON c.code = b.country1 OR c.code = b.country2
JOIN geo_sea gs ON c.code = gs.country
WHERE gs.sea = 'Caribbean Sea'
True cols: Index(['TOTAL_AREA'], dtype='object')
Adj. True cols: Index([], dtype='object')
Predicted cols: Index(['NAME', 'AREA'], dtype='object')
Adj. Predicted cols: Index([], dtype='object')
False: no match found
SELECT c.name AS country_name, SUM(c1.population) AS total_neighbor_population
FROM mondial_Country c
JOIN mondial_Borders b ON c.code = b.country1
JOIN mondial_Country c1 ON b.country2 = c1.code
GROUP B

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SQLAgents\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SQLAgents\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SQLAgents\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI

True cols: Index(['NAME'], dtype='object')
Adj. True cols: Index([], dtype='object')
Predicted cols: Index(['COUNTRY_NAME', 'YEAR', 'POPULATION', 'POPULATION_GROWTH'], dtype='object')
Adj. Predicted cols: Index([], dtype='object')
False: no match found
SELECT c.capital FROM mondial_country c WHERE c.capital NOT IN (SELECT lo.city FROM mondial_located lo )
SELECT city.name FROM city LEFT JOIN located ON city.name = located.city WHERE located.city IS NULL AND city.province = city.name
True cols: Index(['CAPITAL'], dtype='object')
Adj. True cols: Index([], dtype='object')
Predicted cols: Index(['NAME'], dtype='object')
Adj. Predicted cols: Index([], dtype='object')
False: no match found
SELECT p.CAPITAL
FROM MONDIAL_PROVINCE p 
WHERE p.name like '%Andalucía%'
SELECT capital
FROM province
WHERE name = 'Andalucía';
True cols: Index(['CAPITAL'], dtype='object')
Adj. True cols: Index([], dtype='object')
Predicted cols: Index(['CAPITAL'], dtype='object')
Adj. Predicted cols: Index([], dtype='o

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SQLAgents\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SQLAgents\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(SQL_query, con=self.db_connection)
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SQLAgents\..\..\functions\dataset_utils.py:423: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI

SELECT length FROM river WHERE name = 'Tigris'
True cols: Index(['LENGTH'], dtype='object')
Adj. True cols: Index(['LENGTH'], dtype='object')
Predicted cols: Index(['LENGTH'], dtype='object')
Adj. Predicted cols: Index(['LENGTH'], dtype='object')
True: query matches sql (Similarity: 1.0, column match: 1.0)
SELECT name
FROM mondial_airport
WHERE elevation > 1000.0;
SELECT * FROM airport WHERE elevation > 1000;
True cols: Index(['NAME'], dtype='object')
Adj. True cols: Index(['NAME'], dtype='object')
Predicted cols: Index(['IATACODE', 'NAME', 'COUNTRY', 'CITY', 'PROVINCE', 'ISLAND', 'LATITUDE',
       'LONGITUDE', 'ELEVATION', 'GMTOFFSET'],
      dtype='object')
Adj. Predicted cols: Index(['NAME'], dtype='object')
True: query matches sql (Similarity: 1.0, column match: 1.0)
SELECT a.name
FROM mondial_airport a
INNER JOIN mondial_country c
ON c.code = a.country
WHERE c.name = 'United States' AND a.elevation <200;
SELECT * FROM airport WHERE country = 'US' AND elevation < 200
True cols: In

In [18]:
df_fk = pd.DataFrame(results_fk, columns=['experiment', 'simple', 'medium', 'complex', 'total', 'overall_simple', 'overall_medium', 'overall_complex', 'overall_total','input_tokens', 'output_tokens', 'total_tokens','total_cost', 'total_time'])
df_fk['experiment'] = df_fk['experiment'].apply(get_experiment_name)
df_fk = df_fk.sort_values(by='overall_total', ascending=False)
df_fk

,experiment,simple,medium,complex,total,overall_simple,overall_medium,overall_complex,overall_total,input_tokens,output_tokens,total_tokens,total_cost,total_time
1,SQLAgent - MONDIAL_GPT - GPT-3.5 - Passing Dat...,18,16,7,41,0.5455,0.4848,0.2059,0.41,0,0,0,0.0,0:06:40
0,SQLAgent - MONDIAL_GPT - GPT-3.5 - FK,18,13,6,37,0.5455,0.3939,0.1765,0.37,0,0,0,0.0,0:04:52


In [19]:
df_fk.to_csv(TARGET_PATH_FK)  

: 